In [1]:
import hail as hl
hl.init(spark_conf={'spark.driver.memory': '10g'}, tmp_dir='/home/olavur/tmp')

Running on Apache Spark version 2.4.1
SparkUI available at http://hms-beagle-7889d4ff4c-6wxtc:4042
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.61-3c86d3ba497a
LOGGING: writing to /home/olavur/experiments/2020-11-13_fargen1_exome_analysis/fargen-1-exome/notebooks/gnomad_exome_sites/hail-20210326-1333-0.2.61-3c86d3ba497a.log


In [12]:
BASE_DIR = '/home/olavur/experiments/2020-11-13_fargen1_exome_analysis'
RESOURCES_DIR = '/non-fargen/resources'

## Load exome target files

Load the SureSelect Human All Exon V6 UTR target BED file, which is used in the FarGen Phase I exome sequencing.

In [13]:
fargen_interval_ht = hl.import_bed(RESOURCES_DIR + '/sureselect_human_all_exon_v6_utr_grch38/S07604624_Padded.bed', reference_genome='GRCh38')

2021-03-26 13:41:21 Hail: INFO: Reading table without type imputation
  Loading field 'f0' as type str (user-supplied)
  Loading field 'f1' as type int32 (user-supplied)
  Loading field 'f2' as type int32 (user-supplied)
  Loading field 'f3' as type str (user-supplied)


In [127]:
gnomAD_interval_ht = hl.read_table(BASE_DIR + '/data/resources/gnomAD/exome_calling_regions.v1.GRCh38.ht')

## Interval overlaps

### Tests

Temporary variable names for tables.

In [166]:
ht1 = fargen_interval_ht
ht2 = gnomAD_interval_ht

Key one table by start.

In [189]:
ht1 = ht1.annotate(_interval=ht1.interval)

In [167]:
ht2 = ht2.annotate(start=ht2.interval.start)
ht2 = ht2.key_by(ht2.start)

In [190]:
ht1.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Row fields:
    'interval': interval<locus<GRCh38>> 
    'target': str 
    '_interval': interval<locus<GRCh38>> 
----------------------------------------
Key: ['interval']
----------------------------------------


In [191]:
ht2.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Row fields:
    'interval': interval<locus<GRCh38>> 
    'target': str 
    'start': locus<GRCh38> 
----------------------------------------
Key: ['start']
----------------------------------------


In [192]:
ht1.index(ht2.key, all_matches=True)

<ArrayStructExpression of type array<struct{target: str, _interval: interval<locus<GRCh38>>}>>

In [195]:
temp_ht = ht2.select(interval=ht1.index(ht2.key, all_matches=True)._interval)

In [196]:
temp_ht.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Row fields:
    'start': locus<GRCh38> 
    'interval': array<interval<locus<GRCh38>>> 
----------------------------------------
Key: ['start']
----------------------------------------


## Write table to file

In [14]:
if True:
    gnomad_ht.write(BASE_DIR + '/data/resources/gnomAD/gnomad.exomes.r2.1.1.sites.GRCh38.EXOME_HIGH_QUALITY.ht', overwrite=True)

2021-03-26 10:03:02 Hail: INFO: Coerced sorted dataset
2021-03-26 10:06:43 Hail: INFO: wrote table with 16320689 rows in 9997 partitions to /home/olavur/experiments/2020-11-13_fargen1_exome_analysis/data/resources/gnomAD/gnomad.exomes.r2.1.1.sites.GRCh38.EXOME_HIGH_QUALITY.ht
    Total size: 40.13 GiB
    * Rows: 40.13 GiB
    * Globals: 4.59 KiB
    * Smallest partition: 0 rows (21.00 B)
    * Largest partition:  2054 rows (7.57 MiB)
